In [ ]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [ ]:
# location of raw data files
input_path = "/home/jupyter/datasets/raw/"

In [ ]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'INPUTEVENTS_MV.csv',dtype = "object")

In [ ]:
#read D_ITEMS.csv file
df1 = pd.read_csv(filepath_or_buffer=input_path+'D_ITEMS.csv',dtype = "object")

In [ ]:
# map the labels for ITEMD 
df=df.merge(df1[['ITEMID','LABEL']], how='left', left_on='ITEMID', right_on='ITEMID')

In [ ]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

In [ ]:
df.head()

In [ ]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 'columns')

In [ ]:
#Dropping redundant columns
df = df.drop(labels=['STARTTIME','ENDTIME','COMMENTS_EDITEDBY','COMMENTS_CANCELEDBY','COMMENTS_DATE','CGID','ORDERID','LINKORDERID','ISOPENBAG','CONTINUEINNEXTDEPT','CANCELREASON','ORIGINALAMOUNT','ORIGINALRATE'],axis = 'columns')

In [ ]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

In [ ]:
df = df.fillna('NaN')

In [ ]:
# merge ITEMID, VALUE, UOM into single token
df['AMOUNT_VALUE_UOM'] = df['ITEMID']+'-'+df['AMOUNT']+'-'+df['AMOUNTUOM']
df['RATE_VALUE_UOM'] = df['ITEMID']+'-'+df['RATE']+'-'+df['RATEUOM']
df['FLUID_VALUE_UOM'] = 'FLUID'+'-'+df['TOTALAMOUNT']+'-'+df['TOTALAMOUNTUOM']
df['PATIENTWEIGHT'] = 'WEIGHT'+'-'+df['PATIENTWEIGHT']

In [ ]:
df.head()

In [ ]:
# if value if missing then replace merge tokens with 'NaN'
df.loc[df['AMOUNT']=='NaN','AMOUNT_VALUE_UOM'] = 'NaN'
df.loc[df['RATE']=='NaN','RATE_VALUE_UOM'] = 'NaN'
df.loc[df['TOTALAMOUNT']=='NaN','FLUID_VALUE_UOM'] = 'NaN'

In [ ]:
# Drop redundant columns
df = df.drop(labels=['ITEMID','AMOUNT','AMOUNTUOM','RATE','RATEUOM','TOTALAMOUNT','TOTALAMOUNTUOM','PATIENTWEIGHT'],axis = 'columns')

In [ ]:
df.head()

In [ ]:
# Removce any speacial charecters from the free text columns
df['ORDERCATEGORYNAME'] = df['ORDERCATEGORYNAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

df['SECONDARYORDERCATEGORYNAME'] = df['SECONDARYORDERCATEGORYNAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

df['ORDERCOMPONENTTYPEDESCRIPTION'] = df['ORDERCOMPONENTTYPEDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['ORDERCATEGORYDESCRIPTION'] = df['ORDERCATEGORYDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['STATUSDESCRIPTION'] = df['STATUSDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

In [ ]:
# create inputevents_mv by mergeing all the columns on whitespace
df['inputevents_mv'] = df['LABEL']+' '+df['ORDERCATEGORYNAME']+' '+df['SECONDARYORDERCATEGORYNAME']+' '+df['ORDERCOMPONENTTYPEDESCRIPTION']+' '+df['ORDERCATEGORYDESCRIPTION']+' '+df['STATUSDESCRIPTION']+' '+df['AMOUNT_VALUE_UOM']+' '+df['RATE_VALUE_UOM']+' '+df['FLUID_VALUE_UOM']

In [ ]:
# drop redundant columns
df = df.drop(labels=['LABEL','ORDERCATEGORYNAME','SECONDARYORDERCATEGORYNAME','ORDERCOMPONENTTYPEDESCRIPTION','ORDERCATEGORYDESCRIPTION','STATUSDESCRIPTION','AMOUNT_VALUE_UOM','RATE_VALUE_UOM','FLUID_VALUE_UOM'], axis = 'columns')

In [ ]:
df.head()

In [ ]:
# split on white spaceand convert tp list
df['inputevents_mv'] = df['inputevents_mv'].map(lambda x: x.split())

In [ ]:
#Remove 'NaN' from the inputevents_mv
df['inputevents_mv'] = df['inputevents_mv'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [ ]:
df.head(10)

In [ ]:
# create output path
mypath_output = "/home/jupyter/datasets/inputevents_mv/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# export the dataframe to JSON format
df.to_json(mypath_output+"inputevents_mv.json",orient = 'records') 